In [4]:
from __future__ import division
from pyomo.environ import *
from pyomo.core.base.expr import identify_variables
import numpy as np

model = ConcreteModel()


model.n = range(4)

model.x = Var(model.n, within=Binary)

model.c = np.array([1,2,2,1])
# model.a1 = np.matrix([[-1,-1,0,0]])
# model.a2 = np.matrix([[0,0,-1,-1]])
model.a = np.matrix([[-1,-1,0,0],[0,0,-1,-1]])
# model.b1 = np.array([-1])
# model.b2 = np.array([-2])
model.b = np.array([-1,-2])
# model.set1 = range(len(model.b1))
# model.set2 = range(len(model.b2))
model.eqmat = np.matrix([1,1,1,1])
model.eqb = np.array([3])
model.set = range(len(model.b))
consets = [[0],[1]]
varsets = [[0,1],[2,3]]

In [ ]:
model.OBJ = Objective(expr = summation(model.c,model.x))

# model.eq = ConstraintList()
# for i in model.m:
#     model.eq.add(expr = sum(model.x[i,j] for j in model.n) == 1)
    
# model.ineq1 = ConstraintList()
# for i in range(len(model.b1)):
#     model.ineq1.add(expr = sum(model.a1[i,j]*model.x[j] for j in model.n) <= model.b1[i])
# model.ineq2 = ConstraintList()
# for i in range(len(model.b2)):
#     model.ineq2.add(expr = sum(model.a2[i,j]*model.x[j] for j in model.n) <= model.b2[i])
model.eq = ConstraintList()
for i in range(len(model.eqb)):
    print(i)
    model.eq.add(expr = sum(model.eqmat[i,j]*model.x[j] for j in model.n) == model.eqb[i])
model.ineq = ConstraintList()
for i in model.set:
    model.ineq.add(expr = sum(model.a[i,j]*model.x[j] for j in model.n) <= model.b[i])

In [ ]:
model.dual = Suffix(direction=Suffix.IMPORT)
solver = SolverFactory('cplex')
results = solver.solve(model, tee = True)
model.solutions.load_from(results)

In [201]:
# Create the subproblems
# First get a feasible point for each and value it
variables_value = []
for j in range(len(consets)):
    print(varsets[j])
    subprob = ConcreteModel()
    subprob.x = Var(varsets[j],within=Binary)
    subprob.con = ConstraintList()
    for con in consets[j]:
        subprob.con.add(expr = sum(model.a[j,i]*subprob.x[i] for i in varsets[j]) <= model.b[con])
    subprob.OBJ = Objective(expr = 0)
    subprob.dual = Suffix(direction=Suffix.IMPORT)
    solver = SolverFactory('cplex')
    results = solver.solve(subprob, tee = True)
    subprob.solutions.load_from(results)
    mini_list = []
    for v in subprob.component_objects(Var, active=True):
        print("Variable component object",v)
        for index in v:
            variables_value.append(subprob.x[index].value)
#     variables_value.append(mini_list)
# Now value each feasible point
val = []
for j in range(len(consets)):
    val.append(0)
    for i in varsets[j]:
        val[j] += model.c[i]*variables_value[i]
    print(val[j])

[0, 1]

Welcome to IBM(R) ILOG(R) CPLEX(R) Interactive Optimizer 12.8.0.0
  with Simplex, Mixed Integer & Barrier Optimizers
5725-A06 5725-A29 5724-Y48 5724-Y49 5724-Y54 5724-Y55 5655-Y21
Copyright IBM Corp. 1988, 2017.  All Rights Reserved.

Type 'help' for a list of available commands.
Type 'help' followed by a command name for more
information on commands.

CPLEX> Logfile 'cplex.log' closed.
Logfile 'c:\users\daniel~1\appdata\local\temp\tmpgjvgel.cplex.log' open.
CPLEX> Problem 'c:\users\daniel~1\appdata\local\temp\tmpfr2bt2.pyomo.lp' read.
Read time = 0.00 sec. (0.00 ticks)
CPLEX> Problem name         : c:\users\daniel~1\appdata\local\temp\tmpfr2bt2.pyomo.lp
Objective sense      : Minimize
Variables            :       3  [Nneg: 1,  Binary: 2]
Objective nonzeros   :       0
Linear constraints   :       2  [Less: 1,  Equal: 1]
  Nonzeros           :       3
  RHS nonzeros       :       2

Variables            : Min LB: 0.0000000        Max UB: 1.000000       
Objective nonzeros   : M

In [ ]:
val

In [ ]:
# Now create the master problem
master = ConcreteModel()
master.lambda_var = Var(range(len(val)),within=NonNegativeReals)
lambda_sets = [[0],[1]]
master.eq = ConstraintList()
for row in range(len(model.eqb)):
    master.eq.add(expr = 
                 sum( sum( model.eqmat[row,j]*variables_value[j] for j in varsets[i] ) * master.lambda_var[i] for i in range(len(val))) == model.eqb[row]
                 )
master.convexity = ConstraintList()
for i in range(len(consets)):
    print(sum(master.lambda_var[j] for j in lambda_sets[i]))
    master.convexity.add(expr = 
                        sum(master.lambda_var[j] for j in lambda_sets[i]) == 1
                        )
master.OBJ = Objective(expr = 
                      sum(val[j]*master.lambda_var[j] for j in range(len(val)))
                      )

master.dual = Suffix(direction=Suffix.IMPORT)
solver = SolverFactory('cplex')
results = solver.solve(master, tee = True)
master.solutions.load_from(results)

In [ ]:
for v in master.component_objects(Var, active=True):
    print("Variable component object",v)
    for index in v:
        print("   ", index, v[index].value)

In [ ]:
print ("Duals")
from pyomo.core import Constraint
for c in model.component_objects(Constraint, active=True):
    print ("   Constraint",c)
    cobject = getattr(model, str(c))
    print(cobject)
    for index in cobject:
        print(cobject[index])
        print ("      ", index, model.dual[cobject[index]])

In [202]:
def projectBinary(x):
    for row in range(len(x)):
        if x[row] <= 0.5:
            x[row] = 0
        else:
            x[row] = 1
    return x

def projectHyperplane(x,A,b):
    for row in range(A.shape[0]):
        if np.dot(A[row,:],x) - b[row] > 0:
            x = x - ((( (np.dot(np.array(A[row,:]),x))[0] - b[row] )/ ((np.array(np.square(A).sum(axis=1))[row])[0]) ) * np.array(A[row,:].flatten())[0])
    return x

In [204]:
x = np.random.rand(2)
# x = np.array([0.1,0.3])
print(type(x))
A = np.matrix([[-1,-1],[-2,-3]])
b = np.array([-1,-2])
for k in range(1000):
    x = projectHyperplane(x,A,b)
    x = projectBinary(x)
    
print(x)
np.dot(A,x) <= b
print(A.shape[1])

<type 'numpy.ndarray'>
[0. 1.]
2


In [233]:
def find_points(A,b):
    """Finds feasible points for a set of linear constraints"""
    points = []
    for i in range(100):
        x = np.random.rand(A.shape[1])
        x_old = x+1
        while (np.linalg.norm(x-x_old) > 0.00001):
            x_old = x
            x = projectHyperplane(x,A,b)
            x = projectBinary(x)
        points.append(x)
        
    return np.unique(np.array(points),axis=0)

In [235]:
points = find_points(A,b)
print(points)

[[0. 1.]
 [1. 0.]
 [1. 1.]]


In [254]:
# varsets
# model.a
print(model.a[consets[0],varsets[0]])
for i in range(len(varsets)):
    print(i)
    print(find_points(model.a[consets[i],varsets[i]],model.b[consets[i]]))

[[-1 -1]]
0
[[0. 1.]
 [1. 0.]
 [1. 1.]]
1
[[1. 1.]]
